In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.preprocessing import MinMaxScaler
from flask import Flask, request, jsonify, render_template_string
from pyngrok import ngrok
import random
import os

# Authenticate Ngrok
os.system("ngrok authtoken 2v0QxJ87pZ8GmUjq68lhzyrHRAi_5F29hcmA8YWBuk5GDFGPe")

# Initialize Flask App
app = Flask(__name__)
public_url = ngrok.connect(5000).public_url
print("Public URL:", public_url)

def generate_fake_data():
    """Generate fake data for training models."""
    data = {
        'event_id': np.arange(1, 1001),
        'ticket_price': np.random.choice([1000, 2000, 3000, 4000], 1000),
        'tickets_left': np.random.randint(0, 500, 1000),
        'queue_length': np.random.randint(1, 1000, 1000),
        'fraud_score': np.random.uniform(0, 1, 1000),
    }
    return pd.DataFrame(data)

data = generate_fake_data()
scaler = MinMaxScaler()
data[['ticket_price', 'tickets_left', 'queue_length']] = scaler.fit_transform(data[['ticket_price', 'tickets_left', 'queue_length']])

# Ticket Availability Prediction (LSTM)
def build_lstm_model():
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(1, 3)),
        LSTM(50),
        Dense(25, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

lstm_model = build_lstm_model()
X_train = np.array(data[['ticket_price', 'tickets_left', 'queue_length']]).reshape(-1, 1, 3)
y_train = np.random.randint(0, 2, len(data))  # Fake labels for ticket availability
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Queue Time Estimation (Random Forest)
queue_model = RandomForestRegressor()
queue_model.fit(data[['queue_length']], np.random.randint(1, 60, len(data)))  # Predict queue wait time

# Fraud Detection (Isolation Forest)
fraud_model = IsolationForest()
fraud_model.fit(data[['fraud_score']])

# Dynamic Seat Allocation (Q-Learning - Simplified)
class SeatAllocation:
    def __init__(self, total_seats):
        self.seats = np.zeros(total_seats)

    def allocate(self, preference):
        available_seats = np.where(self.seats == 0)[0]
        if len(available_seats) > 0:
            chosen_seat = random.choice(available_seats)
            self.seats[chosen_seat] = 1
            return chosen_seat
        return -1

seat_allocator = SeatAllocation(100)

@app.route("/")
def home():
    return render_template_string('''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Automated Ticket Booking</title>
        <script>
            function startChat() {
                let chatBox = document.getElementById("chatBox");
                chatBox.innerHTML += "<p>Bot: Welcome! Let's book your ticket. What is your maximum budget?</p>";
            }

            function sendResponse() {
                let userInput = document.getElementById("userInput").value;
                let chatBox = document.getElementById("chatBox");
                chatBox.innerHTML += `<p>You: ${userInput}</p>`;

                setTimeout(() => {
                    if (!document.getElementById("budgetSet").value) {
                        chatBox.innerHTML += "<p>Bot: How many tickets do you want to book?</p>";
                        document.getElementById("budgetSet").value = userInput;
                    } else if (!document.getElementById("ticketCountSet").value) {
                        chatBox.innerHTML += "<p>Bot: Please enter your name.</p>";
                        document.getElementById("ticketCountSet").value = userInput;
                    } else if (!document.getElementById("nameSet").value) {
                        chatBox.innerHTML += "<p>Bot: Please enter your address.</p>";
                        document.getElementById("nameSet").value = userInput;
                    } else if (!document.getElementById("addressSet").value) {
                        chatBox.innerHTML += "<p>Bot: If fewer tickets are available, should we book?</p>";
                        document.getElementById("addressSet").value = userInput;
                    } else if (!document.getElementById("distancePrefSet").value) {
                        chatBox.innerHTML += "<p>Bot: If tickets are available at far distances, should we book?</p>";
                        document.getElementById("distancePrefSet").value = userInput;
                    } else {
                        chatBox.innerHTML += "<p>Bot: Thank you! Blocking your funds and booking your ticket when available.</p>";
                    }
                }, 1000);

                document.getElementById("userInput").value = "";
            }
        </script>
    </head>
    <body onload="startChat()">
        <h2>Automated Ticket Booking Chatbot</h2>
        <div id="chatBox" style="border: 1px solid #000; padding: 10px; width: 50%; height: 300px; overflow-y: scroll;"></div>
        <br>
        <input type="text" id="userInput" placeholder="Type your response...">
        <button onclick="sendResponse()">Send</button>
        <input type="hidden" id="budgetSet">
        <input type="hidden" id="ticketCountSet">
        <input type="hidden" id="nameSet">
        <input type="hidden" id="addressSet">
        <input type="hidden" id="distancePrefSet">
    </body>
    </html>
    ''')

if __name__ == "__main__":
    app.run(port=5000)
